# Packages Installs, imports, and presets

In [37]:
#from scapy.all import *
import pandas as pd
#import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os
from transformers import MobileViTV2Model
import torch
import torchmetrics
from torch import optim
import lightning as L
import torch
import torch.nn as nn

In [38]:
INPUT_SIZE = (256,256,3)

# Data preprocessing

In [39]:
def get_attack_directory_attack(path):
  return path[29:]
attacks_directories= [
                      '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1',
                      '20200427_UOWM_IEC104_Dataset_c_se_na_1',
                      '20200428_UOWM_IEC104_Dataset_c_sc_na_1',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS',
                      '20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS',
                      '20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS',
                                            # '20200608_UOWM_IEC104_Dataset_mitm_drop'
                                            ]
get_attack_directory_attack(attacks_directories[0])

'm_sp_na_1_DoS'

In [40]:
df =pd.concat([pd.read_parquet(f"./data/{attack_directory}.parquet") for attack_directory in attacks_directories ])

In [41]:
df.shape

(518377, 1347)

In [42]:
df.index.nunique()

26660

In [43]:
%%time
from tqdm import tqdm
from collections import defaultdict
df=df.sort_values(by=['flow','tv_sec','tv_usec'],axis=0,ascending=True)

TIMEOUT = 15

last_timestamp = defaultdict(lambda: 0)
flow_bin = defaultdict(lambda: 0)
splits = []
for c,r in tqdm(df.iterrows()):
  flow_id = c+'_'+r['label']
  if (r['tv_sec'] - last_timestamp[flow_id] ) >= TIMEOUT:# in the first packet of a flow it's always true
    flow_bin[flow_id] += 1
  splits.append(f"{flow_bin[flow_id]}_{flow_id}")
  last_timestamp[flow_id] = r['tv_sec']
df['flow'] = splits
del splits
df=df.set_index('flow')

518377it [01:46, 4855.05it/s]


CPU times: user 1min 34s, sys: 30 s, total: 2min 4s
Wall time: 1min 59s


In [44]:
df.to_parquet('data/iec104_15.parquet')